In [86]:
#%%
# Importing necessary module
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import glob
import pandas as pd

In [87]:
#%%
# Define constant
BATCH_SIZE = 32
IMAGE_SIZE_WITHOUT_CHANNEL= (608, 736)
IMAGE_SIZE_WITH_CHANNEL = (608, 736, 3)
VALIDATION_RATE = 0.2
OUTPUT_CLASS = 73476
BATCH_SIZE = 32 
EPOCH = 10
DATA_DIR = 'datasets/'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
TEST_DIR = os.path.join(DATA_DIR, 'test')
TRAIN_FILE = os.path.join(DATA_DIR, 'train.csv')
train_file_list = glob.glob(TRAIN_DIR+"/*/*/*/*")

# Loading training data

In [126]:
train_df = pd.read_csv(TRAIN_FILE)
train_df.landmark_id = train_df.landmark_id.astype(str)
train_df["location"] = os.getcwd() +"/"+ TRAIN_DIR + "/"+train_df.id.str[0]+"/"+train_df.id.str[1]+"/"+train_df.id.str[2]+"/"+train_df.id+".jpg"
train_df = train_df[:100000]
train_df
#train_df = train_df.sample(10000, random_state=10)

,id,landmark_id,location
0,17660ef415d37059,1,/Users/golammortuza/workspace/learn/deeplearni...
1,92b6290d571448f6,1,/Users/golammortuza/workspace/learn/deeplearni...
2,cd41bf948edc0340,1,/Users/golammortuza/workspace/learn/deeplearni...
3,fb09f1e98c6d2f70,1,/Users/golammortuza/workspace/learn/deeplearni...
4,25c9dfc7ea69838d,7,/Users/golammortuza/workspace/learn/deeplearni...
...,...,...,...
99995,024640449c8b5379,13197,/Users/golammortuza/workspace/learn/deeplearni...
99996,0b888cd1a25805a2,13197,/Users/golammortuza/workspace/learn/deeplearni...
99997,0f5c2d77654c2cec,13197,/Users/golammortuza/workspace/learn/deeplearni...
99998,1f0429efb4f7cffd,13197,/Users/golammortuza/workspace/learn/deeplearni...


In [129]:
import shutil
didnt = 0
for ids, src in zip(train_df["id"], train_df["location"]):
    dirs = "/Users/golammortuza/Desktop/data/" + ids[0] + "/"+ids[1] + "/"+ids[2] + "/"
    dest = dirs + ids +".jpg"
    try:
        if not os.path.exists(dirs):
            os.makedirs(dirs)
        shutil.copy(src, dest)
    except:
        didnt
        didnt += 1

In [115]:
os.makedirs("/Users/golammortuza/Desktop/data/a/a/a")

In [95]:
gen = ImageDataGenerator(rescale=1./255, validation_split=VALIDATION_RATE)
train_generator = gen.flow_from_dataframe(
    dataframe=train_df,
    directory=TRAIN_DIR,
    x_col="location",
    y_col="landmark_id",
    target_size=IMAGE_SIZE_WITHOUT_CHANNEL,
    seed=42,
    shuffle=True,
    batch_size=BATCH_SIZE,
    subset="training",
    colormode="rgb"
)
validation_generator = gen.flow_from_dataframe(
    dataframe=train_df,
    directory=TRAIN_DIR,
    x_col="location",
    y_col="landmark_id",
    target_size=IMAGE_SIZE_WITHOUT_CHANNEL,
    class_mode="categorical",
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
    subset="validation",
    colormode="rgb"
)

Found 2546 validated image filenames belonging to 2962 classes.
Found 636 validated image filenames belonging to 2962 classes.


# Base model

In [96]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=IMAGE_SIZE_WITH_CHANNEL))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(.3))
model.add(Dense(2962, activation='softmax'))
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_61 (Conv2D)           (None, 606, 734, 16)      448       
_________________________________________________________________
max_pooling2d_60 (MaxPooling (None, 303, 367, 16)      0         
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 301, 365, 64)      9280      
_________________________________________________________________
max_pooling2d_61 (MaxPooling (None, 150, 182, 64)      0         
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 148, 180, 128)     73856     
_________________________________________________________________
max_pooling2d_62 (MaxPooling (None, 74, 90, 128)       0         
_________________________________________________________________
conv2d_64 (Conv2D)           (None, 72, 88, 128)     

In [97]:
model.compile(optimizers.Adam(lr=0.0001, decay=1e-6), loss="categorical_crossentropy", metrics=["categorical_accuracy"])

In [98]:
steps_per_epoch = 2546 // BATCH_SIZE
steps_per_epoch

79

In [99]:
model.fit_generator(train_generator, epochs=EPOCH, validation_data=validation_generator, steps_per_epoch=steps_per_epoch, verbose=1)

Epoch 1/10
79/79 [==============================] - ETA: 0s - loss: 7.9951 - categorical_accuracy: 3.9557e-04

KeyboardInterrupt: 

In [100]:
train_df

,id,landmark_id,location
147,0fbf003f9f98fced,27,0/f/b/0fbf003f9f98fced.jpg
1150265,46642dbd1dbcb95a,147824,4/6/6/46642dbd1dbcb95a.jpg
379656,5306d1a2ec4cafb4,47804,5/3/0/5306d1a2ec4cafb4.jpg
1518657,802bf9e6e3c71b0c,194914,8/0/2/802bf9e6e3c71b0c.jpg
1226810,8d698f60d8f9e5d2,158062,8/d/6/8d698f60d8f9e5d2.jpg
...,...,...,...
954599,c364514f68ddec93,123411,c/3/6/c364514f68ddec93.jpg
488734,e8116ef0c23db3af,62166,e/8/1/e8116ef0c23db3af.jpg
633478,09db88451ea0c4c2,81438,0/9/d/09db88451ea0c4c2.jpg
471664,2157f91ebe244451,60020,2/1/5/2157f91ebe244451.jpg
